<a href="https://colab.research.google.com/github/afairley/ColaboratoryNotebooks/blob/main/FlaxBasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade  -q pip jax jaxlib
!pip install --upgrade -q git+https://github.com/google/flax.git
import jax
from typing import Any, Callable, Sequence
from jax import random, numpy as jnp
import flax
from flax import linen as nn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 MB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
model = nn.Dense(features=5)
key1, key2 = random.split(random.key(0))
x = random.normal(key1, (10,))
params = model.init(key2,x)
jax.tree_util.tree_map(lambda x: x.shape,params)
model.apply(params,x)

n_samples = 20
x_dim = 10
y_dim = 5

nextKey = random.key(0)
k1, k2 = random.split(nextKey)
W = random.normal(k1, (x_dim, y_dim))
b = random.normal(k2,(y_dim,))

true_params = flax.core.freeze({'params':{'bias': b, 'kernel': W}})
key_sample, key_noise = random.split(k1)
x_samples = random.normal(key_sample, (n_samples, x_dim))
y_samples = jnp.dot(x_samples,W) + b + 0.1 *\
 random.normal(key_noise,(n_samples, y_dim))
print('x shape:', x_samples.shape, '; y shape:', y_samples.shape)
print('x:', x_samples, '; y:', y_samples)

In [6]:
@jax.jit
def mean_squared_error(params, model, x_batched, y_batched):
  def squared_error(x, y):
    pred = model.apply(params, x)
    return jnp.inner(y-pred, y-pred) / 2.0
  return jnp.mean(jax.vmap(squared_error)(x_batched, y_batched), axis=0)